In [13]:
os.getcwd()

'/Users/svilenkolev/Downloads'

In [14]:
#!pip install pycromanager

In [15]:
import os
import pandas as pd

In [16]:
#from skimage import data
import napari
#import skimage.measure as meas
import numpy as np
import matplotlib.pyplot as plt
import tifffile
#from pycromanager import Dataset

In [17]:
#import napari
#import numpy as np
#from vispy.color import Colormap
#import tifffile as tiff

In [18]:
#folder_path = 'gut_mask/edof_4set_for_annot'
folder_path = r'/Users/svilenkolev/Library/CloudStorage/OneDrive-NortheasternUniversity/edof_4set_for_annot'
file_list = os.listdir(folder_path)
file_list

['StripA_b393G_b393R_simul_0hr1_devA1_1worm15_edf_4set.tiff',
 'StripA_b393G_b393R_simul_0hr1_devA1_1worm9_edf_4set.tiff',
 'StripA_b393G_b393R_simul_0hr2_devA4_1worm1_edf_4set.tiff',
 'strip1_A_b393G_big393R_simul_2h_r2_manual_devA3_1worm1_edf_4set.tiff',
 'StripA_b393G_b393R_simul_0hr2_devA1_1worm5_edf_4set.tiff',
 'StripA_b393G_b393R_simul_0hr2_devA3_1worm9_edf_4set.tiff',
 'strip1_A_b393G_big393R_simul_2h_r1_manual_devA4_1worm3_edf_4set.tiff',
 'strip1_A_b393G_big393R_simul_3h_r2_manual_devA1_1worm9_edf_4set.tiff',
 'StripA_b393G_b393R_simul_0hr1_devA1_1worm11_edf_4set.tiff',
 'strip1_A_b393G_big393R_simul_2h_r1_manual_devA4_1worm7_edf_4set.tiff',
 'strip1_A_b393G_big393R_simul_2h_r2_manual_devA1_1worm9_edf_4set.tiff',
 'strip1_A_b393G_big393R_simul_2h_r2_manual_devA3_1worm5_edf_4set.tiff',
 'StripA_b393G_b393R_simul_0hr2_devA4_1worm5_edf_4set.tiff',
 'StripA_b393G_b393R_simul_0hr2_devA1_1worm1_edf_4set.tiff',
 'StripA_b393G_b393R_simul_0hr2_devA2_1worm4_edf_4set.tiff',
 'StripA_b3

In [19]:
# class mySet:
#     def __init__(self,path,name,data):
#         self.path = path
#         self.name = name
#         self.data = data
#         self.current_w = None
#         self.num_w = data.shape[0]
#         self.q = np.arange(self.num_w).tolist()
        
#     def get_next_w(self):
#         self.current_w = self.q.pop()
# #         print(self.current_w)
#         return self.data[self.current_w]
    
#     def save_annot(self,annot_stack):
#         tiff.imsave(os.path.join(self.path,self.name+'_{}.tiff'.format(self.current_w)),annot_stack)

class mySet_files_vypng:
    def __init__(self,path,name,file_list):
        self.path = path
        self.name = name
        self.data = None
        self.current_f = None
        self.num_im = len(file_list) #data.shape[0]
        self.q = file_list
        self.current_data = None
        self.completed_files = []
        
    def get_next_im(self):
        self.current_f = self.q.pop()
        self.completed_files.append(self.current_f)
        self.current_data = tifffile.imread(os.path.join(self.path,self.current_f))
        return self.current_data
    
#     def save_annot(self,annot_stack):
#         save_stack = np.concatenate((self.current_data,annot_stack))
#         tiff.imwrite(os.path.join(self.path,self.current_f[:-5]+'_{}.tiff'.format(self.name)),save_stack)
#         print(self.name)
        
def open_stack(viewer,stack):
    """ stack shape should match the name and colorings
        here is for a sigle image"""
    viewer.add_image(
        stack,
        channel_axis=0,
        name=[f"Pos {0}, Channel Brightfield",f"Position {0},Channel edf replacement", f"Position {0}, Channel edf output", f"Position {0}, Channel hmap"],
        colormap=["gray", "green", 'red','magma'],
    )

def save_good_open_next(viewer,myS,df):
    if myS.current_f is None:
        print('opening stack')
        open_stack(viewer,myS.get_next_im())
    else:
        f_name = myS.current_f
        new_data = {'file_name':myS.current_f,'good':True}
        df.loc[len(df)] = new_data
        try:
            viewer.layers.clear()
            open_stack(viewer,myS.get_next_im())
        except IndexError:
            print('finished anotating set')
    
def save_bad_open_next(viewer,myS,df):
    if myS.current_f is None:
        print('opening stack')
        open_stack(viewer,myS.get_next_im())
    else:
        new_data = {'file_name':myS.current_f,'good':False}
        df.loc[len(df)] = new_data
        try:
            viewer.layers.clear()
            open_stack(viewer,myS.get_next_im())
        except IndexError:
            print('finished anotating set')

In [20]:
del df

In [21]:
path = folder_path #os.path.join(os.getcwd(),'gut_mask/edof_4set_for_annot')

myS = mySet_files_vypng(path,'worm_gut_mask',file_list)

df = pd.DataFrame(columns = ['file_name','good'])

viewer = napari.Viewer()

#viewer.bind_key('w', lambda event: add_layer(viewer, 'worm_mask'))
#viewer.bind_key('g', lambda event: add_layer(viewer, 'gut_mask'))
viewer.bind_key('g', lambda event: save_good_open_next(viewer,myS,df))
viewer.bind_key('b', lambda event: save_bad_open_next(viewer,myS,df))

opening stack


In [11]:
df

,file_name,good
0,StripA_b393G_b393R_simul_0hr1_devA2_1worm9_edf...,True
1,StripA_b393G_b393R_simul_0hr2_devA2_1worm5_edf...,True
2,StripA_b393G_b393R_simul_0hr2_devA1_1worm11_ed...,True
3,strip1_A_b393G_big393R_simul_3h_r2_manual_devA...,True
4,StripA_b393G_b393R_simul_0hr2_devA2_1worm1_edf...,True
5,StripA_b393G_b393R_simul_2hr2_devA4_1worm8_edf...,False
6,StripA_b393G_b393R_simul_0hr2_devA1_1worm15_ed...,True
7,strip1_A_b393G_big393R_simul_3h_r2_manual_devA...,False
8,StripA_b393G_b393R_simul_0hr1_devA1_1worm14_ed...,True
9,StripA_b393G_b393R_simul_0hr2_devA1_1worm4_edf...,False
